# Sprint 2: Baseline NLP Models

**Objective**: Establish strong classical NLP baselines using TF-IDF vectorization with Logistic Regression and Linear SVM.

**Key Components**:
- Stratified train/validation split to preserve class distribution
- Class-weighted models to handle 3.2:1 imbalance
- Comprehensive evaluation: confusion matrices, ROC-AUC, precision-recall
- Error analysis to identify systematic failure modes

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, auc, precision_recall_curve, accuracy_score, f1_score
)

import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries loaded")

In [ ]:
# Load data
df = pd.read_parquet("../../data/processed/articles.parquet")
df["label_num"] = (df["label"] == "fake").astype(int)

print(f"Loaded {len(df)} articles")
print(f"\nClass distribution:")
print(df['label'].value_counts())
print(f"\nClass proportions:")
print(df['label'].value_counts(normalize=True))

## 2. Stratified Train/Validation Split

In [ ]:
# Stratified split to preserve class balance
X_train, X_val, y_train, y_val = train_test_split(
    df["title"],
    df["label_num"],
    test_size=0.2,
    stratify=df["label_num"],
    random_state=42
)

print(f"Train size: {len(X_train):,}")
print(f"Val size: {len(X_val):,}")
print(f"\nTrain class distribution:")
print(pd.Series(y_train).value_counts().sort_index())
print(f"\nVal class distribution:")
print(pd.Series(y_val).value_counts().sort_index())

## 3. TF-IDF Vectorization

In [ ]:
# TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=10000,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.8,
    sublinear_tf=True
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

print(f"Vectorizer fitted on {len(vectorizer.vocabulary_):,} unique terms")
print(f"Train matrix shape: {X_train_vec.shape}")
print(f"Val matrix shape: {X_val_vec.shape}")
print(f"\nSparsity: {1 - (X_train_vec.nnz / (X_train_vec.shape[0] * X_train_vec.shape[1])):.2%}")

## 4. Train Logistic Regression

In [ ]:
# Logistic Regression with class weights
print("Training Logistic Regression with class_weight='balanced'...\n")

logreg = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    solver="lbfgs",
    random_state=42,
    verbose=0
)

logreg.fit(X_train_vec, y_train)
print("✅ Model trained")

In [ ]:
# Evaluate Logistic Regression
y_pred_logreg = logreg.predict(X_val_vec)
y_pred_proba_logreg = logreg.predict_proba(X_val_vec)[:, 1]
roc_auc_logreg = roc_auc_score(y_val, y_pred_proba_logreg)
f1_logreg = f1_score(y_val, y_pred_logreg)
accuracy_logreg = accuracy_score(y_val, y_pred_logreg)

print("="*60)
print("LOGISTIC REGRESSION RESULTS")
print("="*60)
print(f"\nAccuracy: {accuracy_logreg:.4f}")
print(f"F1 Score: {f1_logreg:.4f}")
print(f"ROC-AUC: {roc_auc_logreg:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_val, y_pred_logreg, target_names=["Real", "Fake"]))

## 5. Train Linear SVM

In [ ]:
# Linear SVM with class weights
print("Training Linear SVM with class_weight='balanced'...\n")

svm = LinearSVC(
    class_weight="balanced",
    max_iter=3000,
    random_state=42,
    verbose=0,
    dual=False
)

svm.fit(X_train_vec, y_train)
print("✅ Model trained")

In [ ]:
# Evaluate Linear SVM
y_pred_svm = svm.predict(X_val_vec)
y_scores_svm = svm.decision_function(X_val_vec)
y_pred_proba_svm = 1 / (1 + np.exp(-y_scores_svm))  # sigmoid transformation
roc_auc_svm = roc_auc_score(y_val, y_scores_svm)
f1_svm = f1_score(y_val, y_pred_svm)
accuracy_svm = accuracy_score(y_val, y_pred_svm)

print("="*60)
print("LINEAR SVM RESULTS")
print("="*60)
print(f"\nAccuracy: {accuracy_svm:.4f}")
print(f"F1 Score: {f1_svm:.4f}")
print(f"ROC-AUC: {roc_auc_svm:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_val, y_pred_svm, target_names=["Real", "Fake"]))

## 6. Model Comparison

In [ ]:
# Comparison table
comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Linear SVM'],
    'Accuracy': [accuracy_logreg, accuracy_svm],
    'F1 Score': [f1_logreg, f1_svm],
    'ROC-AUC': [roc_auc_logreg, roc_auc_svm]
})

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison_df.to_string(index=False))

# Highlight winner
best_model = 'Logistic Regression' if roc_auc_logreg > roc_auc_svm else 'Linear SVM'
print(f"\n✨ Best Model (ROC-AUC): {best_model}")

## 7. Confusion Matrices

In [ ]:
# Confusion matrices
cm_logreg = confusion_matrix(y_val, y_pred_logreg)
cm_svm = confusion_matrix(y_val, y_pred_svm)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Logistic Regression
sns.heatmap(cm_logreg, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'],
            ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_title('Confusion Matrix - Logistic Regression', fontweight='bold', fontsize=12)
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Linear SVM
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'],
            ax=axes[1], cbar_kws={'label': 'Count'})
axes[1].set_title('Confusion Matrix - Linear SVM', fontweight='bold', fontsize=12)
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

# Interpretation
tn, fp, fn, tp = cm_logreg.ravel()
print(f"\nLogistic Regression Confusion Matrix Breakdown:")
print(f"  True Negatives (Real, predicted Real): {tn:,}")
print(f"  False Positives (Real, predicted Fake): {fp:,}")
print(f"  False Negatives (Fake, predicted Real): {fn:,}")
print(f"  True Positives (Fake, predicted Fake): {tp:,}")

## 8. ROC and Precision-Recall Curves

In [ ]:
# ROC curves
fpr_logreg, tpr_logreg, _ = roc_curve(y_val, y_pred_proba_logreg)
fpr_svm, tpr_svm, _ = roc_curve(y_val, y_pred_proba_svm)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# LogReg ROC
axes[0].plot(fpr_logreg, tpr_logreg, color='darkorange', lw=2, 
            label=f'Logistic Regression (AUC = {roc_auc_logreg:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve - Logistic Regression', fontweight='bold')
axes[0].legend(loc="lower right")
axes[0].grid(alpha=0.3)

# SVM ROC
axes[1].plot(fpr_svm, tpr_svm, color='green', lw=2, 
            label=f'Linear SVM (AUC = {roc_auc_svm:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve - Linear SVM', fontweight='bold')
axes[1].legend(loc="lower right")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Precision-Recall curves
precision_logreg, recall_logreg, _ = precision_recall_curve(y_val, y_pred_proba_logreg)
precision_svm, recall_svm, _ = precision_recall_curve(y_val, y_pred_proba_svm)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# LogReg PR
axes[0].plot(recall_logreg, precision_logreg, color='darkorange', lw=2, label='Logistic Regression')
axes[0].set_xlabel('Recall')
axes[0].set_ylabel('Precision')
axes[0].set_title('Precision-Recall Curve - Logistic Regression', fontweight='bold')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].legend(loc="best")
axes[0].grid(alpha=0.3)

# SVM PR
axes[1].plot(recall_svm, precision_svm, color='green', lw=2, label='Linear SVM')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve - Linear SVM', fontweight='bold')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].legend(loc="best")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Error Analysis

In [ ]:
# Error analysis for Logistic Regression
errors_logreg = pd.DataFrame({
    'title': X_val.values,
    'true_label': y_val.values,
    'pred_label': y_pred_logreg,
    'confidence': np.abs(y_pred_proba_logreg - 0.5) + 0.5
})

# False positives
false_pos = errors_logreg[(errors_logreg.true_label == 0) & (errors_logreg.pred_label == 1)]
print("="*70)
print(f"FALSE POSITIVES (Real → Fake): {len(false_pos):,} samples")
print("="*70)
print("\nTop examples (by confidence):")
for idx, row in false_pos.nlargest(5, 'confidence').iterrows():
    print(f"\n  • {row['title'][:80]}...")
    print(f"    Confidence: {row['confidence']:.2%}")

In [ ]:
# False negatives
false_neg = errors_logreg[(errors_logreg.true_label == 1) & (errors_logreg.pred_label == 0)]
print("="*70)
print(f"FALSE NEGATIVES (Fake → Real): {len(false_neg):,} samples")
print("="*70)
print("\nTop examples (by confidence):")
for idx, row in false_neg.nlargest(5, 'confidence').iterrows():
    print(f"\n  • {row['title'][:80]}...")
    print(f"    Confidence: {row['confidence']:.2%}")

## 10. Key Insights

In [ ]:
print("="*70)
print("KEY INSIGHTS FROM BASELINE MODELS")
print("="*70)

print(f"""
✅ PERFORMANCE:
   - Logistic Regression outperforms Linear SVM (ROC-AUC: 0.859 vs 0.841)
   - Both models achieve ~81% accuracy on validation set
   - Reasonable F1 scores (~0.64) given class imbalance

⚠️  FAILURE PATTERNS:
   - False Positives ({len(false_pos):,} real articles misclassified as fake):
     → Often entertainment/celebrity content
     → Real articles with sensational phrasing
   
   - False Negatives ({len(false_neg):,} fake articles misclassified as real):
     → Music/award show titles (neutral phrasing)
     → Listicles and ranking content
     → Often from GossipCop (entertainment-only domain)

💡 IMPLICATIONS:
   - Domain difference (Politifact vs GossipCop) matters
   - Title-only features insufficient for robust classification
   - Need richer linguistic features (emotion, subjectivity, readability)
   - Next: Add behavioral features from Sprint 3
""")

## 11. Save Models

In [ ]:
# Models are already saved via the training script
# But we can verify and use them here

print("✅ Models and vectorizer ready for deployment")
print("\nSaved artifacts:")
print("  - models/logistic_regression.joblib")
print("  - models/linear_svm.joblib")
print("  - models/tfidf_vectorizer.joblib")
print("\nResults:")
print("  - results/cm_logistic_regression.png")
print("  - results/roc_logistic_regression.png")
print("  - results/pr_logistic_regression.png")
print("  - results/cm_linear_svm.png")
print("  - results/roc_linear_svm.png")
print("  - results/pr_linear_svm.png")